## Установка зависимостей

In [1]:
# !pip install torch transformers scikit-learn datasets
# !pip install 'accelerate>=0.26.0'

## Подготовка данных

Будем заниматься задачей детекции токсичных комментариев. Данные у нас находятся в папке `data`. В файле `normal.txt` лежат обычные комментарии, в `toxic.txt` -- токсичные. Для тренировки модели надо собрать два разных набора: `X` и `y`. `X` Будет входом для модели. `y` будет переменной, которую хотим предсказать. В нашем случае `X` -- комментарий, а `y` -- метка 1 (комментарий токсичный) или 0 (комментарий обычный).

In [2]:
# Комментарии у нас разделены двумя переносами строки
# То есть чтобы получить список комментариев, нужно разделить даные по двум переносам строки
# Открываем файл, читаем содержимое и разделяем по двум переносам
normal_comments = (
    open("data/normal.txt")
    .read()
    .split("\n\n")
)

In [3]:
toxic_comments = (
    open("data/toxic.txt")
    .read()
    .split("\n\n")
)

In [4]:
# Посмотрим, сколько у нас обычных и токсичных комментариев...
len(normal_comments), len(toxic_comments)

(8971, 4473)

In [5]:
# ...и сделаем для них метки!
normal_labels = [0] * len(normal_comments)
toxic_labels  = [1] * len(toxic_comments)

In [6]:
# И наконец соберем наши данные
X = normal_comments + toxic_comments
y = normal_labels + toxic_labels
len(X), len(y)

(13444, 13444)

In [7]:
X[13], y[13]

('Моя знакомая, лет 10 как не курит. Это всё при том, что муж как курил так и курит в квартире.(в вытяжку принудительную и мощную на кухне). Тянет до сих пор её иногда, сны снятся как вновь закурила, до сих пор то в жар бросает, то в холод из за тяги. Старается не выпивать алкогольные напитки, потому что желание взять снова сигарету, адски просто нарастает. Занимается йогой, пробежки по утрам, да и без дела не сидит чтоб хоть как то отвлечься..',
 0)

In [8]:
X[12000], y[12000]

(', мат. Мои нежные, девственные чувства оскорблены.', 1)

In [9]:
# Последнее, что здесь надо сделать -- это разделить данные на тренировочную и тестовую выборки.
# Это нужно для того, чтобы понять, как модель работает на данных, которые она не видела во время обучения.
# Для этого нам надо
# 1) Перемешать наши данные 
#    (ведь сейчас у нас в самом начале всегда обычные комменты, а в конце -- токсичные.
#    Модель нас не поймет во время тренировки)
# 2) Разделить их в какой-то пропорции
# Сделать всё это можно так:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,   # Доля тестовой части
    random_state=42, # Делаем рандом нерандомным, чтобы результаты всегда были одинаковые
    shuffle=True     # И перемешиваем всё
)

In [10]:
len(X_train), len(X_test), len(y_train), len(y_test)

(10755, 2689, 10755, 2689)

## Подготовка модели

В современном NLP обычно не нужно учить свою собственную модель прям с нуля. Существуют модели, предобученные (pretrained) для работы с естественным языком. Они долго учатся на огромных разнообразных массивах текста (речь о терабайтах), чтобы понять язык глобально. Идея в том, что потом эти знания можно переложить (transfer) на более узкую задачу, для которых данных сильно меньше. Мы именно это и сделаем: возьмем одну такую модель и дообучим (finetune) так, чтобы она понимала _концепт токсичности_.

Существуют разные базовые модели, которые могут генерировать текст, переводить на разные языки, генерировать текст по картинке или картинку по тексту. В рамках нашей задачи нам нужна модель, которая _понимает_ язык, то есть принимает на вход текст и извлекает из него смысл. Для такой задачи используются модели класса "кодировщик" (encoder). Модель-кодировщик BERT или что-то вдохновленное им отлично подойдет.

Мы возьмем [RuModernBERT от deepvk](https://huggingface.co/deepvk/RuModernBERT-base). Это самая свежая модель BERT для русского языка. Полный список всех моделей для разных задач и языков можно [посмотреть тут](https://huggingface.co/) порывшисть по поисковику.

In [11]:
import torch

In [12]:
# Сначала надо определить, на каком устройстве будем учить и использовать модель.
# Использовать видеокарту если она есть, иначе -- центральный процессор.
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/smertlove/sandbox/hse/dl/Bert-Toxicity/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
model_id = "deepvk/RuModernBERT-base"

In [15]:
# Мы не можем просто дать модели на вход текст. Нужно как-то уметь преобразровывать его в числа.
# Поэтому все модели состоят из, собственно, модели,
# и токенизатора -- штуки, которая разбивает входной текст на кусочки знакомым модели образом.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at deepvk/RuModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
tokens = tokenizer("Я — Батай, а ты — Лакан, Пойдём скорей плясать канкан.")

In [17]:
# Вот так это выглядит
tokens['input_ids']

[50281,
 1128,
 683,
 3064,
 5642,
 18,
 227,
 147,
 115,
 1599,
 683,
 5245,
 2367,
 18,
 926,
 299,
 34417,
 35801,
 9856,
 24523,
 436,
 951,
 1099,
 20,
 50282]

In [18]:
# А тут видим, что токенизатор для модели добавляет в текст всякие прикольные штуки.
tokenizer.decode(tokens['input_ids'])

'[CLS]Я — Батай, а ты — Лакан, Пойдём скорей плясать канкан.[SEP]'

In [19]:
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import numpy as np

In [20]:
# Тут готовим модель к дообучению.
# Фактически тут диктант, который просто нужно уметь написать.
# Всей интеллектуальной деятельностью, как ни странно, мы занимались выше.
# (если это теперь увидит кто-то, меня на работу никуда не возьмут xdd)

train_dataset = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})

test_dataset = Dataset.from_dict({
    'text': X_test,
    'label': y_test
})

tokenize_function = lambda examples: tokenizer(
    examples['text'],
    padding=True,
    truncation=True,
    max_length=512,
)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

model_params = list(model.base_model.parameters())

# Замораживаем часть слоев модели
for param in model_params:
    param.requires_grad = False

# Размораживаем только последние слои
for param in model_params[-6:]:  # Последние 6 слоев
    param.requires_grad = True

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="steps",            # Оцениваем по шагам
    eval_steps=500,                   # Каждые 500 шагов
    logging_strategy="steps",         # Логируем по шагам
    logging_steps=100,                # Каждые 100 шагов
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_dir='./logs',
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

trainer.train()

Map: 100%|██████████| 2689/2689 [00:00<00:00, 7018.05 examples/s]
/tmp/ipykernel_23792/1013032888.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Step,Training Loss,Validation Loss
500,0.231800,0.221361
1000,0.205900,0.210403
1500,0.180000,0.202807
2000,0.157400,0.201453


TrainOutput(global_step=2019, training_loss=0.24476644306740233, metrics={'train_runtime': 1138.7741, 'train_samples_per_second': 28.333, 'train_steps_per_second': 1.773, 'total_flos': 1.099455190170624e+16, 'train_loss': 0.24476644306740233, 'epoch': 3.0})

Ура! Что-то натренировалось.
Цифры в табличке сверху показывают что модель каждый раз училась чему-то новому (Training loss  Validation loss стабильно падают).
Возможно, стоило бы поучить ее еще немного, но для примера хватит.

## Теперь можно пользоваться моделью)

In [25]:
from transformers import pipeline

In [ ]:
# Для удобства тут собираем модель в пайплайн, чтобы в одну команду проведить все операции
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [27]:
result = classifier("Какая же ты падла блять")
result

[{'label': 'LABEL_1', 'score': 1.0}]

In [28]:
result = classifier("Я люблю маму и папу")
result

[{'label': 'LABEL_0', 'score': 0.9736777544021606}]

По-хорошему здесь еще должен быть замер качества и метрики accuracy, F1, roc-auc и пр., но это пока опустим. Цель данной тетрадки -- показать, что всё не так страшно, как может выглядеть на первый взгляд :)